In [1]:
from transfer_learning_publication.models import ModelFactory
from transfer_learning_publication.data import CaravanDataSource, LSHDataModule
from transfer_learning_publication.transforms import PipelineBuilder, Log, ZScore
from transfer_learning_publication.models import ModelFactory, ModelEvaluator
from transfer_learning_publication.models.tide import LitTiDE
from transfer_learning_publication.callbacks import HFUploadCallback
import lightning as pl
from litmodels.integrations import LightningModelCheckpoint
import torch
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping, Callback, LearningRateMonitor, RichProgressBar
import matplotlib.pyplot as plt
import joblib
import os
from huggingface_hub import upload_file

## Transform data and sink to disk

In [2]:
ts_features = ["temperature_2m_max",
            "temperature_2m_min",
            "potential_evaporation_sum_FAO_PENMAN_MONTEITH",
            "potential_evaporation_sum_ERA5_LAND",
            "temperature_2m_mean",
            "total_precipitation_sum",
            "streamflow",
            "streamflow_was_filled",
            "sin_day_of_year",
            "cos_day_of_year"]

static_features = ["p_mean",
            "area",
            "ele_mt_sav",
            "high_prec_dur",
            "frac_snow",
            "high_prec_freq",
            "slp_dg_sav",
            "cly_pc_sav",
            "aridity_ERA5_LAND",
            "aridity_FAO_PM",]

stages = ["train", "val", "test"]

In [3]:
# ts_preprocessing_pipelines = (
#     PipelineBuilder(group_identifier="gauge_id")
#     .add_per_basin(Log(), columns=["streamflow"])
#     .add_global(ZScore(), columns=["temperature_2m_max",
#             "temperature_2m_min",
#             "potential_evaporation_sum_FAO_PENMAN_MONTEITH",
#             "potential_evaporation_sum_ERA5_LAND",
#             "temperature_2m_mean",
#             "total_precipitation_sum",
#             "streamflow"])
#     .build()
# )

# static_preprocessing_pipelines = (
#     PipelineBuilder(group_identifier="gauge_id")
#     .add_global(ZScore(), columns=static_features)
#     .build()
# )

# for stage in stages:
#     caravan = CaravanDataSource(f"/Users/cooper/Desktop/CARAVAN_CLEAN/{stage}", region="tajikkyrgyz")
#     gauge_ids = caravan.list_gauge_ids()
#     ts_data = caravan.get_timeseries(
#         gauge_ids[:3], 
#         columns=ts_features
#     )
#     static_data = caravan.get_static_attributes(
#         gauge_ids[:3], 
#         columns=static_features
#     )

#     if stage == "train":
#         ts_transformed = ts_preprocessing_pipelines.fit_transform(ts_data.collect())
#         static_transformed = static_preprocessing_pipelines.fit_transform(static_data.collect())
#     else:
#         ts_transformed = ts_preprocessing_pipelines.transform(ts_data.collect())
#         static_transformed = static_preprocessing_pipelines.transform(static_data.collect())

#     caravan.write_timeseries(ts_transformed, f"/Users/cooper/Desktop/first-test/{stage}", overwrite=False)
#     caravan.write_static_attributes(static_transformed, f"/Users/cooper/Desktop/first-test/{stage}", overwrite=False)

#     print(f"Completed {stage} stage")

In [4]:
# # Dump the fitted pipelines
# joblib.dump(ts_preprocessing_pipelines, "/Users/cooper/Desktop/first-test/ts_pipeline.joblib")
# joblib.dump(static_preprocessing_pipelines, "/Users/cooper/Desktop/first-test/static_pipeline.joblib")

## Create datamodule and instantiate model for training

In [5]:
config_path = "/Users/cooper/Desktop/transfer-learning-publication/configs/first_run.yaml"

In [6]:
datamodule = LSHDataModule(config_path=config_path)

model_factory = ModelFactory()
model = model_factory.create_from_config(config_path=config_path)

In [7]:
trainer = pl.Trainer(
    max_epochs=1,
    accelerator="auto",
    devices=1,
    logger=True,
    # callbacks=[
    #     ModelCheckpoint(
    #         monitor="val_loss", 
    #         mode="min", 
    #         save_top_k=1,
    #         dirpath="checkpoints",
    #         filename="best-{epoch:02d}-{val_loss:.2f}"
    #     ),
    #     HFUploadCallback(
    #         repo_id="CooperBigFoot/test-repo",
    #         repo_type="model",
    #         path_in_repo="checkpoints/{filename}",
    #         private=False
    #     ),
    #     EarlyStopping(
    #         monitor="val_loss", 
    #         mode="min", 
    #         patience=5, 
    #         verbose=True
    #     ),
    #     RichProgressBar() 
    # ],
    enable_progress_bar=True,
    log_every_n_steps=1
)

trainer.fit(model=model, datamodule=datamodule)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/cooper/Desktop/transfer-learning-publication/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default

  | Name      | Type      | Params | Mode 
------------------------------------------------
0 | criterion | 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/cooper/Desktop/transfer-learning-publication/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:433: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.
/Users/cooper/Desktop/transfer-learning-publication/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:433: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


In [8]:
models_and_datamodules = {
    "tide": (model, datamodule),
}

evaluator = ModelEvaluator(
    models_and_datamodules=models_and_datamodules,
    trainer_kwargs={"accelerator": "cpu"}
)

results = evaluator.test_models(cache_dir="/Users/cooper/Desktop/transfer-learning-publication/data/cache/first_run/", force_recompute=True)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/cooper/Desktop/transfer-learning-publication/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
/Users/cooper/Desktop/transfer-learning-publication/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:433: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │   0.046355124562978745    │
└───────────────────────────┴───────────────────────────┘

In [9]:
print(results.summary())

shape: (1, 5)
┌────────────┬───────────┬──────────┬───────────────┬───────────┐
│ model_name ┆ n_samples ┆ n_basins ┆ output_length ┆ has_dates │
│ ---        ┆ ---       ┆ ---      ┆ ---           ┆ ---       │
│ str        ┆ i64       ┆ i64      ┆ i64           ┆ bool      │
╞════════════╪═══════════╪══════════╪═══════════════╪═══════════╡
│ tide       ┆ 6146      ┆ 3        ┆ 10            ┆ true      │
└────────────┴───────────┴──────────┴───────────────┴───────────┘


In [10]:
print(results.raw_data.head(10))

shape: (10, 7)
┌────────────┬───────────────┬───────────┬────────────┬─────────────┬───────────────┬──────────────┐
│ model_name ┆ group_identif ┆ lead_time ┆ prediction ┆ observation ┆ issue_date    ┆ prediction_d │
│ ---        ┆ ier           ┆ ---       ┆ ---        ┆ ---         ┆ ---           ┆ ate          │
│ str        ┆ ---           ┆ i64       ┆ f64        ┆ f64         ┆ datetime[μs]  ┆ ---          │
│            ┆ str           ┆           ┆            ┆             ┆               ┆ datetime[μs] │
╞════════════╪═══════════════╪═══════════╪════════════╪═════════════╪═══════════════╪══════════════╡
│ tide       ┆ tajikkyrgyz_1 ┆ 1         ┆ 0.610679   ┆ 0.546317    ┆ 2017-04-30    ┆ 2017-05-01   │
│            ┆ 5013          ┆           ┆            ┆             ┆ 22:00:00      ┆ 22:00:00     │
│ tide       ┆ tajikkyrgyz_1 ┆ 2         ┆ 0.616001   ┆ 0.537825    ┆ 2017-04-30    ┆ 2017-05-02   │
│            ┆ 5013          ┆           ┆            ┆             ┆ 22:00:

In [15]:
results.filter(model_name="tide", basin_id="tajikkyrgyz_15013")

model_name,group_identifier,lead_time,prediction,observation,issue_date,prediction_date
str,str,i64,f64,f64,datetime[μs],datetime[μs]
"""tide""","""tajikkyrgyz_15013""",1,0.610679,0.546317,2017-04-30 22:00:00,2017-05-01 22:00:00
"""tide""","""tajikkyrgyz_15013""",2,0.616001,0.537825,2017-04-30 22:00:00,2017-05-02 22:00:00
"""tide""","""tajikkyrgyz_15013""",3,0.530818,0.546317,2017-04-30 22:00:00,2017-05-03 22:00:00
"""tide""","""tajikkyrgyz_15013""",4,0.47633,0.586322,2017-04-30 22:00:00,2017-05-04 22:00:00
"""tide""","""tajikkyrgyz_15013""",5,0.643392,0.602654,2017-04-30 22:00:00,2017-05-05 22:00:00
…,…,…,…,…,…,…
"""tide""","""tajikkyrgyz_15013""",6,0.336791,0.266443,2022-12-20 23:00:00,2022-12-26 23:00:00
"""tide""","""tajikkyrgyz_15013""",7,0.310448,0.269767,2022-12-20 23:00:00,2022-12-27 23:00:00
"""tide""","""tajikkyrgyz_15013""",8,0.318505,0.273016,2022-12-20 23:00:00,2022-12-28 23:00:00
